In [155]:
import os
import spacy
from spacy.tokens import Doc
import sddk
import pandas as pd
import re
import matplotlib.pyplot as plt
import json
import shutil
from gensim.models import KeyedVectors
import pandas as pd
import itertools
from collections import Counter
import nltk
import re
import requests

import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.oauth2 import service_account # based on google-auth library

In [3]:
nlp = spacy.load('la_core_web_lg')

In [11]:
metadata_table_long = pd.read_json("../data/metadata_table_long.json")
metadata_table_long.head(5)

,Author,Full title,In,Year,Place,Publisher/Printer,Era,Form/Genre,Discipline/Content,Original,...,ids,id,date_min,date_max,filename,file_year,sents_n,words_n,disciplines_list,science_class
0,"Achrelius, Daniel",Scientiarum magnes recitatus publice anno 1690...,None,1690,[Turku],Wall,17th century,Oration,"Mathematics, Astronomy/Astrology/Cosmography, ...",Scientiarum magnes(Google Books),...,[705665],705665,1690.0,1690.0,"Achrelius,_Daniel_-_Scientiarum_magnes__Turku_...",1690.0,518,6685,"[Mathematics, Astronomy/Astrology/Cosmography,...",mixture
1,"Acidalius, Valens","Ad Iordanum Brunum Nolanum, Italum","Poematum Iani Lernutii, Iani Gulielmi, Valenti...",1603,"Liegnitz, Wrocław","Albert, David",17th century,Panegyric poem,Astronomy/Astrology/Cosmography,Ad Iordanum Brunum (1603)(CAMENA)Ad Iordanum B...,...,[801745],801745,1603.0,1603.0,Janus_Lernutius_et_al__-_Poemata__Liegnitz_160...,1603.0,6137,66744,[Astronomy/Astrology/Cosmography],exact_sciences
2,"Acosta, José de",De natura novi orbis libri duo et De promulgat...,None,1589,Salamanca,Guillelum Foquel,16th century,Monograph,"Astronomy/Astrology/Cosmography, Geography/Car...",De natura novi orbis(Biodiversity Heritage Lib...,...,[713323],713323,1589.0,1589.0,"Acosta,_José_de_-_De_natura_novi_orbis__Salama...",1589.0,9040,135979,"[Astronomy/Astrology/Cosmography, Geography/Ca...",exact_sciences
3,"Adam, Melchior","Vitae Germanorum medicorum, qui saeculo superi...",None,1620,Heidelberg,"Rosa, Geyder",17th century,Biography,Medicine,Vitae Germanorum medicorum(MDZ)Alternative lin...,...,[693148],693148,1620.0,1620.0,"Adam,_Melchior_-_Vitae_Germanorum_medicorum__H...",1620.0,16258,153868,[Medicine],life_sciences
4,"Addison, Joseph",Ad insignissimum virum dominum Thomam Burnettu...,"Examen poeticum duplex, sive, Musarum anglican...",1698,London,Richard Wellington I.,17th century,Panegyric poem,Meteorology/Earth sciences,Ad Burnettum sacrae theoriae telluris auctorem...,...,[769230],769230,1698.0,1698.0,Examen_poeticum_duplex__London_1698_pdf.txt,1698.0,3297,38011,[Meteorology/Earth sciences],exact_sciences


In [13]:
ids = list(metadata_table_long["ids"].apply(lambda x: eval(x)[0]))
ids[:10]

[705665,
 801745,
 713323,
 693148,
 769230,
 732899,
 1479057,
 670589,
 598104,
 598104]

In [4]:
filtered_vocab_df = pd.read_json("../data/filtered_vocab_df.json")
filtered_vocab_df.head(20)

,word,1501-1550,1551-1600,1601-1650,1651-1700,mean,in_lila_embeddings,in_lasla,in_operamaiora,transl
5018,dico,42831,120825,41832,72221,69427.25,True,True,True,"say, call, tell"
6155,omnis,33152,108530,46009,66387,63519.50,True,True,True,
2515,facio,33323,109486,40134,53953,59224.00,True,True,True,"do, make, handle"
4040,pars,32591,90667,43593,51249,54525.00,True,True,True,part
6769,habeo,30884,87225,38386,57791,53571.50,True,True,True,"have, hold, possess, consider, think"
364,possum,23869,79572,36898,72892,53307.75,True,True,True,"be able, can"
897,res,27869,68790,27473,49014,43286.50,True,True,True,suddenly
4810,suus,23716,58975,33267,52441,42099.75,True,True,True,"his own, her own, its own, their own"
2392,uideo,21389,60143,28355,54288,41043.75,True,True,True,
1790,magnus,23446,61213,29708,39211,38394.50,True,True,True,"big, great"


In [14]:
vocab_list = filtered_vocab_df["word"].tolist()

In [5]:
len(filtered_vocab_df)

6332

In [6]:
s = sddk.cloudSession(provider="sciencedata.dk", shared_folder_name="TOME/DATA/NOSCEMUS", owner="kase@zcu.cz")

connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/TOME/DATA/NOSCEMUS/


In [7]:
target_folder_name = "noscemus_spacyjsons_v1"
jsonfiles_list = s.list_filenames(target_folder_name, "json")
jsonfiles_list[:10]

['1031760.json',
 '1085290.json',
 '1285853.json',
 '1285854.json',
 '1285855.json',
 '1285856.json',
 '1365811.json',
 '1370560.json',
 '1378359.json',
 '1424044.json']

In [8]:
len(jsonfiles_list)

1007

In [16]:
def filename_to_doc(filename, dir=target_folder_name):
    doc_json = s.read_file(dir + "/" + filename, "dict")
    doc = Doc(nlp.vocab).from_json(doc_json)
    return doc

In [17]:
# a test doc
doc = filename_to_doc(jsonfiles_list[0])

In [19]:
sent = list(doc.sents)[151]
sent

Quâ in re si quid profecerimus, non alia sanè ratio nobis viam aperuit, quàm vera & legitima spiritus humant humiliatio.

In [101]:
[(t.lemma_, t.text) for t in sent if (t.pos_ in ["NOUN", "PROPN", "ADJ", "VERB"]) & (t.lemma_ in vocab_list)]

[('res', 're'),
 ('sanè', 'sanè'),
 ('ratio', 'ratio'),
 ('uia', 'viam'),
 ('aperio', 'aperuit'),
 ('quàm', 'quàm'),
 ('uerus', 'vera'),
 ('legitimus', 'legitima'),
 ('spiritus', 'spiritus')]

In [23]:
t.id

NameError: name 't' is not defined

In [ ]:
vocab_instances = {}

In [137]:
def get_vocab_data(id, vocab_list):
    doc_vocab_instances = []
    filename = str(id) + ".json"
    doc = filename_to_doc(filename)
    for sent_n, sent in enumerate(doc.sents):
        doc_vocab_instances.extend([(t.lemma_, t.text, id, sent_n, t.pos_) for t in sent if (t.pos_ in ["NOUN", "PROPN", "ADJ", "VERB"]) & (t.lemma_ in vocab_list)])
    return doc_vocab_instances

In [138]:
doc = filename_to_doc(str(ids[0]) + ".json")

In [139]:
doc_vocab_instances = get_vocab_data(ids[0], vocab_list)
len(doc_vocab_instances)

3176

In [141]:
list(doc.sents)[0]

Scientiarum Magnes.

In [142]:
doc_vocab_instances[:100]

[('magnes', 'Magnes', 705665, 0, 'PROPN'),
 ('optimi', 'Optimi', 705665, 1, 'PROPN'),
 ('par', 'pari', 705665, 2, 'ADJ'),
 ('ardus', 'ardore', 705665, 2, 'NOUN'),
 ('primus', 'primum', 705665, 2, 'ADJ'),
 ('suus', 'suum', 705665, 2, 'ADJ'),
 ('facio', 'facta', 705665, 4, 'VERB'),
 ('series', 'series', 705665, 4, 'NOUN'),
 ('longus', 'longissima', 705665, 4, 'ADJ'),
 ('res', 'rerum', 705665, 4, 'NOUN'),
 ('dominus', 'Domino', 705665, 7, 'NOUN'),
 ('annexus', 'Annexorum', 705665, 7, 'ADJ'),
 ('pax', 'Pax', 705665, 8, 'NOUN'),
 ('secura', 'Secura', 705665, 8, 'ADJ'),
 ('omnis', 'Omnia', 705665, 10, 'ADJ'),
 ('consiliis', 'Consiliis', 705665, 11, 'NOUN'),
 ('decursus', 'decursu', 705665, 13, 'NOUN'),
 ('perego', 'peregit', 705665, 13, 'VERB'),
 ('oratio', 'Oratione', 705665, 13, 'NOUN'),
 ('vigor', 'vigor', 705665, 14, 'NOUN'),
 ('alacritas', 'alacritas', 705665, 15, 'NOUN'),
 ('multus', 'plures', 705665, 15, 'ADJ'),
 ('ingenium', 'ingenii', 705665, 15, 'NOUN'),
 ('foetus', 'foetus', 70566

In [143]:
def update_vocab(vocab_dict, doc_data):
    for lemma, wordform, doc_id, sent_id, pos  in doc_data:
        if lemma not in vocab_dict:
            vocab_dict[lemma] = {}
        if wordform not in vocab_dict[lemma]:
            vocab_dict[lemma][wordform] = []
        # append the tuple to the list of the appropriate wordform of the lemma
        vocab_dict[lemma][wordform].append((doc_id, sent_id, pos))
    return vocab_dict

In [190]:
vocab_instances = {}
for id in ids:
    doc_vocab_instances = get_vocab_data(id, vocab_list)
    vocab_instances = update_vocab(vocab_instances, doc_vocab_instances)

file has not been found; check filename and path.


TypeError: 'NoneType' object is not subscriptable

In [195]:
vocab_instances["magnes"]

{'Magnes': [(705665, 0, 'PROPN'),
  (705665, 133, 'PROPN'),
  (705665, 140, 'PROPN'),
  (705665, 246, 'PROPN'),
  (705665, 357, 'PROPN'),
  (705665, 450, 'PROPN'),
  (658534, 10367, 'PROPN'),
  (839338, 3526, 'PROPN'),
  (839338, 5677, 'PROPN'),
  (718447, 156, 'PROPN'),
  (695815, 4147, 'PROPN'),
  (695815, 4149, 'PROPN'),
  (835557, 8655, 'PROPN'),
  (835557, 15138, 'PROPN'),
  (835557, 15141, 'PROPN'),
  (835557, 24143, 'PROPN'),
  (835557, 24561, 'NOUN'),
  (835557, 24562, 'PROPN'),
  (1424044, 891, 'PROPN'),
  (1424044, 1531, 'PROPN'),
  (1031760, 2860, 'PROPN'),
  (1031760, 2861, 'PROPN'),
  (1031760, 2861, 'PROPN'),
  (1031760, 2862, 'PROPN'),
  (1031760, 2862, 'PROPN'),
  (1031760, 2863, 'PROPN'),
  (1031760, 3044, 'PROPN'),
  (1031760, 3875, 'PROPN'),
  (1031760, 4072, 'PROPN'),
  (1031760, 4344, 'PROPN'),
  (1031760, 4487, 'PROPN'),
  (1031760, 5109, 'NOUN'),
  (901145, 89754, 'PROPN'),
  (901145, 89883, 'PROPN'),
  (733808, 2347, 'PROPN'),
  (605035, 39396, 'PROPN'),
  (7067

In [209]:
ids_covered_list = []
for key in vocab_instances.keys():
    wordforms_dict = vocab_instances[key]
    for wordform in wordforms_dict.keys():
        ids_covered_list.extend([tup[0] for tup in wordforms_dict[wordform]])

In [210]:
ids_covered = list(set(ids_covered_list))
len(ids_covered)

282

In [204]:
type(ids_covered[0])

int

In [ ]:
ids_failed = []
for id in ids:
    if id not in ids_covered:
        try:
            doc_vocab_instances = get_vocab_data(id, vocab_list)
            vocab_instances = update_vocab(vocab_instances, doc_vocab_instances)
        except:
            ids_failed.append(id)

file has not been found; check filename and path.
file has not been found; check filename and path.


In [207]:
len(vocab_instances.keys())

6202

In [208]:
with open("../data/large_data/vocab_instances.json", 'w') as f:
    json.dump(vocab_instances, f)

In [ ]:
# check whether we have sentences for all documents:
f_sents_ready = os.listdir("../data/large_data/sents_text")
len(f_sents_ready)

In [161]:
### choose a lemma for
lemma = "portendo"

In [ ]:
id = ids[0]
metadata = metadata_table_long[metadata_table_long["id"] == id].squeeze()
metadata

In [171]:
metadata_table_long.columns

Index(['Author', 'Full title', 'In', 'Year', 'Place', 'Publisher/Printer',
       'Era', 'Form/Genre', 'Discipline/Content', 'Original',
       'Digital sourcebook', 'Description', 'References', 'Cited in',
       'How to cite this entry', 'Internal notes', 'Of interest to',
       'Transkribus text available', 'Written by', 'Library and Signature',
       'ids', 'id', 'date_min', 'date_max', 'filename', 'file_year', 'sents_n',
       'words_n', 'disciplines_list', 'science_class'],
      dtype='object')

In [180]:
def read_specific_line(filename, line_number):
    with open(filename, 'r', encoding='utf-8') as f:
        return next(itertools.islice(f, line_number, None)).strip()
    
def get_contexts_for_lemma(lemma):
    contexts_data = vocab_instances[lemma]
    contexts_texts = []
    for wordform in contexts_data.keys():
        for tup in contexts_data[wordform]:
            try:   
                filename = "../data/large_data/sents_text/{}.txt".format(str(tup[0]))
                metadata = metadata_table_long[metadata_table_long["id"] == tup[0]].squeeze()
                context_text = read_specific_line(filename, tup[1])
                contexts_texts.append({"lemma" : lemma, "wordform": wordform, "pos" : tup[2], "sentence_text" : context_text, "doc_id" : tup[0], "sentence_i" : tup[1], "author" : metadata["Author"], "title" : metadata["Full title"], "year" : metadata["file_year"]})
            except:
                pass
    contexts_texts_df = pd.DataFrame(contexts_texts)
    contexts_texts_df.sort_values("year", ascending=False)
    return contexts_texts_df

In [181]:
lemma = "honor"
contexts_texts_df = get_contexts_for_lemma(lemma)

In [183]:
len(contexts_texts_df)

122

In [184]:
contexts_texts_df.tail(10)

,lemma,wordform,pos,sentence_text,doc_id,sentence_i,author,title,year
112,honor,honoribus,NOUN,plissimisque honoribus affecit.,693148,4952,"Adam, Melchior","Vitae Germanorum medicorum, qui saeculo superi...",1620.0
113,honor,honoribus,NOUN,quae nam 1569. tinuos quindecim annos cum fove...,693148,8341,"Adam, Melchior","Vitae Germanorum medicorum, qui saeculo superi...",1620.0
114,honor,honoribus,NOUN,"um, quem aliquot annos gessit praeclarè; ampli...",693148,8623,"Adam, Melchior","Vitae Germanorum medicorum, qui saeculo superi...",1620.0
115,honor,honoribus,NOUN,ibique biennium commoratus bingam redit magnos...,693148,9815,"Adam, Melchior","Vitae Germanorum medicorum, qui saeculo superi...",1620.0
116,honor,honoribus,NOUN,quàm equus elegans & commodus quam in parte sa...,693148,12177,"Adam, Melchior","Vitae Germanorum medicorum, qui saeculo superi...",1620.0
117,honor,honoribus,NOUN,"honoribus defunctus est, & Ioauni Vincentis Pi...",693148,12334,"Adam, Melchior","Vitae Germanorum medicorum, qui saeculo superi...",1620.0
118,honor,honoribus,NOUN,Funecta & Bayerorum est familia multis aetatib...,693148,12886,"Adam, Melchior","Vitae Germanorum medicorum, qui saeculo superi...",1620.0
119,honor,Honor,NOUN,"Membro Honor., Soc.",1479057,1127,"Agardh, Carl A.",Aphorismi botanici,1817.0
120,honor,Honor,NOUN,"Membro Honor., Soc.",1479057,2079,"Agardh, Carl A.",Aphorismi botanici,1817.0
121,honor,Honor,NOUN,"Membro Honor., Soc.",1479057,2588,"Agardh, Carl A.",Aphorismi botanici,1817.0
